In [6]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC,LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
import pickle
import pandas as pd

pd.options.display.max_rows = 35 
pd.options.display.max_columns = None

In [7]:
def get_pickles(): 
    x_train = pickle.load(open(f'../../data/processed/pickles/cluster_x_train.p', 'rb'))
    x_test = pickle.load(open(f'../../data/processed/pickles/cluster_x_test.p', 'rb'))
    y_train = pickle.load(open(f'../../data/processed/pickles/cluster_y_train.p', 'rb'))
    y_test = pickle.load(open(f'../../data/processed/pickles/cluster_y_test.p', 'rb'))
    X = pickle.load(open('../../data/processed/pickles/cluster_X.p', 'rb'))
    y = pickle.load(open('../../data/processed/pickles/cluster_y.p', 'rb'))

    return (x_train, x_test, y_train, y_test), (X,y)

(x_train, x_test, y_train, y_test), (X,y) = get_pickles()

print(f'Original X: {X.shape}\tOriginal y: {y.shape}')
print(f'Train X: {x_train.shape}\tTrain y: {y_train.shape}')
print(f'Test X: {x_test.shape}\tTest y: {y_test.shape}')





standard = StandardScaler() 
x_train[['amount_tsh', 'gps_height', 
         'population', 'time_passed']] = standard.fit_transform(x_train[['amount_tsh', 
                                                                         'gps_height', 'population', 'time_passed']])
unique_clusters = x_train.cluster.unique() 
complete_x = x_train.append(x_test, ignore_index = False)
complete_y = y_train.append(y_test, ignore_index = False)
complete_df = complete_x
complete_df['target'] = complete_y.target.values

y_train = y_train.target.values.ravel()
y_test = y_test.target.values.ravel()
y = y.target.values.ravel()

Original X: (57247, 55)	Original y: (57247, 1)
Train X: (42935, 54)	Train y: (42935, 1)
Test X: (14312, 54)	Test y: (14312, 1)


In [14]:
models = {'log': LogisticRegression(max_iter = 2000),
           'knn': KNeighborsClassifier(),
          'dt': DecisionTreeClassifier(), 'Gaussian': GaussianNB(), 'LDA': LinearDiscriminantAnalysis(),
          'rf': RandomForestClassifier(),
        }
#create stacked model
stack_m = [] 
for model, m in models.items(): 
    stack_m.append((model, m))
stack_model = StackingClassifier(estimators = stack_m, final_estimator = LogisticRegression(), cv = 5)
models['stacked'] = stack_model


stack_model = models['stacked']

def get_stacked(train, test): 
    pass

In [15]:



best_score = 0 
best_i = 0
r_back = [i for i in range(8,53)]
pbar = tqdm(r_back)
for i in pbar: 
    pbar.set_description(f'Testing {i} Features || Best AUC: {best_score} || Best n: {best_i}')
    rfe = RFE(stack_model, n_features_to_select = i, step = 2)

    rfe.fit(x_train,y_train)
    pred = rfe.predict(x_test)
    roc_auc = roc_auc_score(y_test, pred)
    if roc_auc > best_score: 
        best_score = roc_auc 
        best_i = i
    #masks for columns that are important
        column_masks = rfe.support_

        orig_columns = x_train.columns
        new_columns = [x for x,y in zip(orig_columns, column_masks) if y == True]
        pickle.dump(new_columns, open('../../models/Stacked_Columns.p', 'wb'))

        


Testing 8 Features || Best AUC: 0 || Best n: 0:   0%|                                                                                                              | 0/45 [00:49<?, ?it/s]


RuntimeError: The classifier does not expose "coef_" or "feature_importances_" attributes